# Speech Understanding | Programming Assignment 2

    Ayush Abrol B20AI052

---

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import os
import wandb

### Question 2

**Goal**: The goal of speech separation is to estimate individual speaker signals from their mixture, where the source signals may be overlapped with each other entirely or partially.

**Tasks**: 

    1. Generate the LibriMix dataset by combining two speakers from the LibriSpeech dataset, focusing solely on the LibriSpeech test clean partition. 

    2. Partition the resulting LibriMix dataset into a 70-30 split for training and testing purposes. Evaluate the performance of the pre-trained SepFormer on the testing set, employing scale-invariant signal-to-noise ratio improvement (SISNRi) and signal-to-distortion ratio improvement (SDRi) as metrics. 

    3. Fine-tune the SepFormer model using the training set and report its performance on the test split of the LibriMix dataset.

    4. Provide observations on the changes in performance throughout the experiment.